You are building an AI system to be able to look at customer reviews and do some complex analysis. for each review get ChatGPT to do the following:

  - Summarize the review. The summary should be at most 3 lines.
  - Highlight both the positives and negatives
  - Display the overall sentiment of the review (positive, negative, neutral)
  - Display a list of 3 - 5 emotions expressed by the customer in the review
  - If the sentiment is positive or neutral write an email and thank them for the review
  - If the sentiment is negative apologize and write an email with an appropriate response

Try to get the response in a nice structured format using an output parser

In [1]:
# Importing the relevant libraries
import os
from dotenv import load_dotenv
import langchain
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import display, Markdown
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_community.callbacks import get_openai_callback
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache


In [2]:
# loading the environment variables
load_dotenv()

True

In [3]:
# Interfacing with chat models
chatgpt = ChatOpenAI(model = "gpt-4o-mini",temperature = 0)

In [10]:
set_llm_cache(InMemoryCache())

In [4]:
## Reviews
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

In [5]:
# Output Parser
class  output_parser(BaseModel):
    Summary: str = Field(description="Summarize the review. The summary should be at most 3 lines")
    positives: str = Field(description="Extract the lines that talk about the positives of the product")
    negatives: str = Field(description="Extract the lines that talk about the negatives of the product")
    sentiment: str = Field(description="Display the overall sentiment of the review (positive, negative, neutral)")
    emotions: str = Field(description="Display a list of 3 - 5 emotions expressed by the customer in the review")
    email: str = Field(description="If the sentiment is positive or neutral write an email and thank them for the review and if the sentiment is negative apologize and write an email with an appropriate response")

parser = PydanticOutputParser(pydantic_object = output_parser)
    

In [7]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"Summary": {"description": "Summarize the review. The summary should be at most 3 lines", "title": "Summary", "type": "string"}, "positives": {"description": "Extract the lines that talk about the positives of the product", "title": "Positives", "type": "string"}, "negatives": {"description": "Extract the lines that talk about the negatives of the product", "title": "Negatives", "type": "string"}, "sentiment": {"description": "Display the overall sentiment of the review (positive, negative, neutral)", "title": "Sentimen

In [17]:
# Prompt

prompt = """Act as an AI assistant for a customer service representative at an online store.
            The representative has received a review from a customer who purchased a product from the store.
            Also remember to write a detailed email response for the email field based on these conditions:
               - email should be addressed to Dear Customer and signed with Service Agent
               - thank them if the review is positive or neutral
               - apologize if the review is negative
            
            Format the output as per the formatting guidelines provided below:
                                {Formatting_Guidelines}

                                The review is as follows:
                                        {Review}
        """

template = PromptTemplate(
                          template = prompt,
                          input_variables= ["Review"],
                          partial_variables={"Formatting_Guidelines": parser.get_format_instructions()},
                        )


In [18]:
# Creating the chain
chain = (
    template
       |
    chatgpt
       |
     parser
)

inputs = [{"Review" : Review} for Review in reviews]
print(inputs)

[{'Review': "\n    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.\n    The sound quality is impressively clear with just the right amount of bass.\n    It's also waterproof, which tested true during a recent splashing incident.\n    Though it's compact, the volume can really fill the space.\n    The price was a bargain for such high-quality sound.\n    Shipping was also on point, arriving two days early in secure packaging.\n    "}, {'Review': "\n    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.\n    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,\n    enhancing my gaming experience significantly. Price-wise, it's quite competitive,\n    and I feel like I got a good deal. The delivery was swift, and it came well-protected,\n    ensuring no damage during transport.\n    "}, {'Review': "\n    Ordered a set of wireless earbuds for running, and they've been a 

In [19]:
response = chain.map().invoke(inputs)

In [20]:
for i in response:
    print(i.model_dump())
    print("\n")

{'Summary': 'The customer is very pleased with the Bluetooth speaker, highlighting its sound quality, waterproof feature, and value for money. They also appreciated the prompt shipping.', 'positives': "The sound quality is impressively clear with just the right amount of bass. It's also waterproof, which tested true during a recent splashing incident. The price was a bargain for such high-quality sound. Shipping was also on point, arriving two days early in secure packaging.", 'negatives': '', 'sentiment': 'positive', 'emotions': 'satisfaction, excitement, appreciation', 'email': "Dear Customer,\n\nThank you so much for your wonderful review! We're thrilled to hear that you are enjoying your Bluetooth speaker and that it met your expectations in terms of sound quality and waterproof features. We also appreciate your kind words about our shipping service. Your satisfaction is our top priority, and we look forward to serving you again in the future!\n\nBest regards,\nService Agent"}


{'

In [21]:
response[0].model_dump()

{'Summary': 'The customer is very pleased with the Bluetooth speaker, highlighting its sound quality, waterproof feature, and value for money. They also appreciated the prompt shipping.',
 'positives': "The sound quality is impressively clear with just the right amount of bass. It's also waterproof, which tested true during a recent splashing incident. The price was a bargain for such high-quality sound. Shipping was also on point, arriving two days early in secure packaging.",
 'negatives': '',
 'sentiment': 'positive',
 'emotions': 'satisfaction, excitement, appreciation',
 'email': "Dear Customer,\n\nThank you so much for your wonderful review! We're thrilled to hear that you are enjoying your Bluetooth speaker and that it met your expectations in terms of sound quality and waterproof features. We also appreciate your kind words about our shipping service. Your satisfaction is our top priority, and we look forward to serving you again in the future!\n\nBest regards,\nService Agent"}

In [23]:
import pandas as pd

pd.DataFrame(response.model_dump() for response in response)

,Summary,positives,negatives,sentiment,emotions,email
0,The customer is very pleased with the Bluetoot...,The sound quality is impressively clear with j...,,positive,"satisfaction, excitement, appreciation","Dear Customer,\n\nThank you so much for your w..."
1,The customer is very satisfied with their new ...,The keys have a satisfying click and the LED c...,,positive,"satisfaction, excitement, appreciation","Dear Customer,\n\nThank you for your wonderful..."
2,The customer is disappointed with the wireless...,They did arrive on time.,"The sound constantly cuts out, and the fit is ...",negative,"disappointment, frustration, dissatisfaction","Dear Customer,\n\nThank you for your feedback ..."
3,The customer expressed disappointment with the...,It did arrive promptly.,"It wobbles with the slightest touch, and the a...",negative,"disappointment, frustration, dissatisfaction, ...","Dear Customer,\n\nThank you for taking the tim..."
4,The customer had a negative experience with th...,,"Struggles with tougher foods, incredibly noisy...",negative,"frustration, disappointment, dissatisfaction, ...","Dear Customer,\n\nThank you for taking the tim..."
